In [17]:
%matplotlib inline

In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [19]:
device = ''
gpu_num = 3
if torch.cuda.is_available():
    device = 'cuda:'+str(gpu_num)
else:
    device = 'cpu'

print('now using '+ device +' as device')

now using cuda:3 as device


In [20]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [21]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

ハイパーパラメータ
- number of epochs
- batch size
- learning rate

In [22]:
learning_rate = 0.01
batch_size = 64
epochs = 5

損失関数：Loss Function

一般的な損失関数としては、回帰タスクではnn.MSELoss、分類タスクではnn.NLLLoss(Negative Log Likelihood)が使用される。
nn.CrossEntropyLossは、nn.LogSoftmaxとnn.NLLLossを結合した損失関数である。
モデルが出力するlogit値をnn.CrossEntropyLossに与えて正規化し、予測誤差を求める。

In [23]:
# loss functionの初期化
loss_function = nn.CrossEntropyLoss()

最適化アルゴリズム：Optimization Algorithms

最適化アルゴリズムは、最適化プロセスの具体的な手続きである。
最適化のロジックは全てoptimizerオブジェクト内に隠蔽される。
今回は確率的勾配降下法：SGD(Stochastic Gradient Descent)を用いる。
ただし、最適化関数にはADAMやRMSPropなど様々ある。
訓練したいモデルパラメータをoptimizerに登録し、合わせて学習率をハイパーパラメータとして渡すことで初期化を行う。

In [24]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

いざ、最適化

In [25]:
def train_loop(dataloader, model, loss_function, optimizer):
    size = len(dataloader.dataset)
    for batch, (x,y) in enumerate(dataloader):
        # 予測と損失の計算
        pred = model(x)
        loss = loss_function(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad() # モデルのパラメータの勾配をリセットする
        loss.backward() # バックプロパゲーションを実行する
        optimizer.step() # 各パラメータの勾配を使用してパラメータの値を調整する

        if batch % 100 == 0:
            loss = loss.item()
            current = batch * len(x)
            print('loss:', loss, current)
        
def test_loop(dataloader, model, loss_function):
    size = len(dataloader.dataset)
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for x,y in dataloader:
            pred = model(x)
            test_loss += loss_function(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= size
    correct /= size
    print('accuracy:', 100*correct)
    print('avg loss:', test_loss)

In [26]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print('epoch:', t+1)
    train_loop(train_dataloader, model, loss_function, optimizer)
    test_loop(test_dataloader, model, loss_function)

print('Done!')

epoch: 1
loss: 2.3058481216430664 0
loss: 2.275848865509033 6400
loss: 2.151771306991577 12800
loss: 2.0560784339904785 19200
loss: 2.014932155609131 25600
loss: 1.6336414813995361 32000
loss: 1.8182488679885864 38400
loss: 1.5035771131515503 44800
loss: 1.5724382400512695 51200
loss: 1.541049838066101 57600
accuracy: 47.48
avg loss: 0.023777528738975526
epoch: 2
loss: 1.4821146726608276 0
loss: 1.622977375984192 6400
loss: 1.232366919517517 12800
loss: 1.4794665575027466 19200
loss: 1.4643677473068237 25600
loss: 1.230865240097046 32000
loss: 1.5441426038742065 38400
loss: 1.2364217042922974 44800
loss: 1.3607491254806519 51200
loss: 1.392218828201294 57600
accuracy: 54.86
avg loss: 0.02105268778204918
epoch: 3
loss: 1.2368872165679932 0
loss: 1.4158728122711182 6400
loss: 1.0439715385437012 12800
loss: 1.3498764038085938 19200
loss: 1.3443604707717896 25600
loss: 1.1205943822860718 32000
loss: 1.3884501457214355 38400
loss: 1.1371403932571411 44800
loss: 1.2686280012130737 51200
loss